# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846877701070                   -0.70    4.2
  2   -7.852333059202       -2.26       -1.53    1.0
  3   -7.852646087373       -3.50       -2.52    3.0
  4   -7.852646679173       -6.23       -3.35    2.8
  5   -7.852646686143       -8.16       -4.83    1.0
  6   -7.852646686726       -9.23       -5.27    4.0
  7   -7.852646686730      -11.46       -6.02    1.8
  8   -7.852646686730      -12.66       -7.45    2.0
  9   -7.852646686730   +    -Inf       -7.66    2.2
 10   -7.852646686730      -14.75       -8.50    2.2
 11   -7.852646686730   +    -Inf       -9.52    2.2
 12   -7.852646686730   +  -15.05      -10.70    2.2
 13   -7.852646686730   +  -14.75      -10.84    1.8
 14   -7.852646686730      -14.35      -11.69    2.2
 15   -7.852646686730   +  -14.45      -13.20    2.2


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846858900089                   -0.70           4.2
  2   -7.852528851304       -2.25       -1.62   0.80    5.0
  3   -7.852610275726       -4.09       -2.80   0.80    5.2
  4   -7.852646558031       -4.44       -3.55   0.80    7.2
  5   -7.852646683625       -6.90       -4.48   0.80    8.5
  6   -7.852646686667       -8.52       -5.08   0.80   10.5
  7   -7.852646686725      -10.24       -6.22   0.80   11.2
  8   -7.852646686730      -11.30       -7.23   0.80   13.8
  9   -7.852646686730      -13.59       -7.65   0.80   15.5
 10   -7.852646686730      -14.45       -9.38   0.80   15.5
 11   -7.852646686730   +  -14.57       -9.69   0.80   19.0
 12   -7.852646686730   +  -14.27      -10.83   0.80   19.2
 13   -7.852646686730      -14.57      -11.93   0.80   22.5
 14   -7.852646686730   +    -Inf      -11.86   0.80   24.2
 15   -7.852646686730      -14.75      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.376375e+01     3.753496e+00
 * time: 0.5171298980712891
     1     1.218460e+00     2.081003e+00
 * time: 0.7062849998474121
     2    -1.538046e+00     2.295943e+00
 * time: 0.7297458648681641
     3    -3.439329e+00     1.751309e+00
 * time: 0.763495922088623
     4    -4.592073e+00     1.641736e+00
 * time: 0.7973048686981201
     5    -6.356601e+00     1.161460e+00
 * time: 0.831387996673584
     6    -7.160275e+00     8.872473e-01
 * time: 0.8652348518371582
     7    -7.274009e+00     1.086670e+00
 * time: 0.8884708881378174
     8    -7.502782e+00     9.754670e-01
 * time: 0.9117879867553711
     9    -7.727123e+00     2.286766e-01
 * time: 0.9451658725738525
    10    -7.774111e+00     1.386305e-01
 * time: 0.968390941619873
    11    -7.798688e+00     8.850380e-02
 * time: 0.9920339584350586
    12    -7.820389e+00     9.358192e-02
 * time: 1.015496015548706
    13    -7.831588e+00     6.423664e-02
 * time: 1.03885197639465

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846822143262                   -0.70    3.8
  2   -7.852311251185       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686576                   -2.55
  2   -7.852646686730       -9.81       -5.77
  3   -7.852646686730   +    -Inf      -11.59
  4   -7.852646686730      -14.75      -16.01
|ρ_newton - ρ_scf|  = 5.008151922650764e-14
|ρ_newton - ρ_scfv| = 2.6275389757732753e-14
|ρ_newton - ρ_dm|   = 7.500416861852792e-10
